In [0]:
use mgiglia.dev_matthew_giglia_price_transparency;

FROM (FROM (FROM (FROM index_json_bronze |> 
SELECT file_metadata, ingest_time, try_parse_json(value) as variant_col) 
,LATERAL variant_explode(variant_col) |>
SELECT file_metadata, ingest_time, key, value |>
PIVOT (first(value) FOR key IN ('reporting_entity_name', 'reporting_entity_type', 'reporting_structure')) |>
SELECT file_metadata, ingest_time, reporting_entity_name::string, reporting_entity_type::string, reporting_structure)
,LATERAL variant_explode(reporting_structure) as reporting_structure
,LATERAL variant_explode(reporting_structure.value) as reporting_structure_value |>
SELECT file_metadata, ingest_time, reporting_entity_name, reporting_entity_type, reporting_structure_value.key as file_type, reporting_structure_value.value)
,LATERAL variant_explode(value) as file_desc
,LATERAL variant_explode(file_desc.value) as file_desc_value |>
SELECT file_metadata, ingest_time, reporting_entity_name, reporting_entity_type, file_type, file_desc.key, file_desc.pos, file_desc_value.key as file_desc_value_key, file_desc_value.pos as file_desc_value_pos, file_desc_value.value as file_desc_value_value|>
SELECT * EXCEPT (key, file_desc_value_pos) |>
PIVOT (first(file_desc_value_value) FOR file_desc_value_key IN ('description','plan_id_type','plan_id','plan_market_type','plan_name','location')) |> 
select file_metadata, ingest_time, reporting_entity_name, reporting_entity_type, file_type, description::string, plan_id_type::string, plan_id::string, plan_market_type::string, plan_name::string, location::string;
